In [4]:
pip install pyproj

   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   --- ------------------------------------ 0.5/6.3 MB 4.2 MB/s eta 0:00:02
   -------- ------------------------------- 1.3/6.3 MB 3.2 MB/s eta 0:00:02
   ------------- -------------------------- 2.1/6.3 MB 3.5 MB/s eta 0:00:02
   ------------------ --------------------- 2.9/6.3 MB 3.6 MB/s eta 0:00:01
   ----------------------- ---------------- 3.7/6.3 MB 3.8 MB/s eta 0:00:01
   -------------------------- ------------- 4.2/6.3 MB 3.6 MB/s eta 0:00:01
   ------------------------------ --------- 4.7/6.3 MB 3.4 MB/s eta 0:00:01
   ----------------------------------- ---- 5.5/6.3 MB 3.3 MB/s eta 0:00:01
   -------------------------------------- - 6.0/6.3 MB 3.3 MB/s eta 0:00:01
   ---------------------------------------- 6.3/6.3 MB 3.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests

# API 키 설정
APP_KEY = 'zLgji89bB19xrdLEokv2e1nXzfXK9haz6i6s6ASE'  # 발급받은 API 키 입력

# 보행자 경로 찾기 함수
def get_pedestrian_route(start_point, end_point, pass_list=None):
    """
    보행자 경로 계산
    start_point: (위도, 경도) 형태의 출발지 좌표
    end_point: (위도, 경도) 형태의 도착지 좌표
    pass_list: 경유지 좌표 목록 (최대 5개)
    """
    url = f'https://apis.openapi.sk.com/tmap/routes/pedestrian?version=1&format=json'
    
    # 요청 데이터 구성
    request_data = {
        'startX': start_point[1],  # 경도
        'startY': start_point[0],  # 위도
        'endX': end_point[1],      # 경도
        'endY': end_point[0],      # 위도
        'reqCoordType': 'WGS84GEO',
        'resCoordType': 'EPSG3857',
        'startName': '출발지',
        'endName': '도착지'
    }
    
    # 경유지가 있는 경우 추가
    if pass_list:
        pass_points = []
        for point in pass_list:
            pass_points.append(f"{point[1]},{point[0]}")  # 경도,위도 형식
        request_data['passList'] = '_'.join(pass_points)
    
    # 헤더 설정
    headers = {
        'appKey': APP_KEY
    }
    
    # API 요청
    response = requests.post(url, json=request_data, headers=headers)
    
    # 응답 확인
    if response.status_code == 200:
        return response.json()
    else:
        print(f"API 요청 실패: {response.status_code}")
        return None

# 초를 hh:mm:ss 형식으로 변환하는 함수
def format_time(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

# 사용 예시
if __name__ == "__main__":
    # 서울역과 광화문 좌표 (위도, 경도)
    seoul_station = (37.5559, 126.9723)
    gwanghwamun = (37.5759, 126.9768)
    
    # 경로 계산
    route_data = get_pedestrian_route(seoul_station, gwanghwamun)
    
    if route_data:
        # 총 거리와 소요 시간 출력
        total_distance = route_data['features'][0]['properties']['totalDistance']
        total_time = route_data['features'][0]['properties']['totalTime']
        
        # 거리 및 시간 출력 (시간은 hh:mm:ss 형식으로)
        print(f"총 거리: {total_distance}m")
        print(f"예상 소요 시간: {format_time(total_time)}")
        
        # 경로 좌표 추출 (시각화나 추가 처리를 위해)
        coordinates = []
        for feature in route_data['features']:
            if feature['geometry']['type'] == 'LineString':
                for coord in feature['geometry']['coordinates']:
                    # EPSG3857에서 WGS84로 좌표 변환 필요
                    # 여기서는 간단히 좌표만 추출
                    coordinates.append(coord)
        
        print(f"경로 좌표 개수: {len(coordinates)}")

총 거리: 2750m
예상 소요 시간: 00:36:39
경로 좌표 개수: 148


In [ ]:
import folium
import pandas as pd
import requests
from pyproj import Transformer
import webbrowser  # 웹브라우저 모듈 추가
import os  # 파일 경로 관리를 위한 모듈

# API 키 설정
APP_KEY = 'zLgji89bB19xrdLEokv2e1nXzfXK9haz6i6s6ASE'  # 발급받은 API 키 입력

# 보행자 경로 계산 함수
def get_pedestrian_route(start_point, end_point):
    """
    보행자 경로 계산
    start_point: (위도, 경도) 형태의 출발지 좌표
    end_point: (위도, 경도) 형태의 도착지 좌표
    """
    url = f'https://apis.openapi.sk.com/tmap/routes/pedestrian?version=1&format=json'
    
    request_data = {
        'startX': start_point[1],  # 경도
        'startY': start_point[0],  # 위도
        'endX': end_point[1],      # 경도
        'endY': end_point[0],      # 위도
        'reqCoordType': 'WGS84GEO',
        'resCoordType': 'EPSG3857',
        'startName': '출발지',
        'endName': '도착지'
    }
    
    headers = {
        'appKey': APP_KEY
    }
    
    response = requests.post(url, json=request_data, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"API 요청 실패: {response.status_code}")
        return None

# EPSG:3857에서 WGS84(EPSG:4326)로 좌표 변환 함수
def convert_coordinates(x, y):
    transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326")
    lat, lon = transformer.transform(x, y)
    return lat, lon

# 경로 시각화 함수
def visualize_route(route_data, start_point, end_point):
    # 지도 생성 (시작점 중심)
    m = folium.Map(location=start_point, zoom_start=15)
    
    # 출발지, 도착지 마커 추가
    folium.Marker(
        location=start_point,
        popup='출발지',
        icon=folium.Icon(color='green', icon='play')
    ).add_to(m)
    
    folium.Marker(
        location=end_point,
        popup='도착지',
        icon=folium.Icon(color='red', icon='stop')
    ).add_to(m)
    
    # 경로 좌표 추출 및 변환
    coordinates = []
    for feature in route_data['features']:
        if feature['geometry']['type'] == 'LineString':
            for coord in feature['geometry']['coordinates']:
                # EPSG3857에서 WGS84로 좌표 변환
                lat, lon = convert_coordinates(coord[0], coord[1])
                coordinates.append([lat, lon])
    
    # 경로 그리기
    folium.PolyLine(
        locations=coordinates,
        color='blue',
        weight=5,
        opacity=0.7,
        tooltip='보행자 경로'
    ).add_to(m)
    
    # 지도 경계 최적화
    if coordinates:
        df = pd.DataFrame(coordinates, columns=['Lat', 'Lon'])
        sw = df[['Lat', 'Lon']].min().values.tolist()
        ne = df[['Lat', 'Lon']].max().values.tolist()
        m.fit_bounds([sw, ne])
    
    # 현재 작업 디렉토리에 절대 경로로 파일 저장
    output_file = os.path.abspath('pedestrian_route.html')
    m.save(output_file)
    
    # 웹브라우저로 파일 열기
    webbrowser.open('file://' + output_file)
    
    return m

# 사용 예시
if __name__ == "__main__":
    # 서울역과 광화문 좌표 (위도, 경도)
    seoul_station = (37.5559, 126.9723)
    gwanghwamun = (37.5759, 126.9768)
    
    # 경로 계산
    route_data = get_pedestrian_route(seoul_station, gwanghwamun)
    
    if route_data:
        # 경로 시각화 및 브라우저에서 열기
        visualize_route(route_data, seoul_station, gwanghwamun)
        print("경로 시각화 완료! 웹브라우저에서 지도를 확인하세요.")
    else:
        # API 요청이 실패한 경우 기존 파일 열기
        existing_file = os.path.abspath('pedestrian_route.html')
        if os.path.exists(existing_file):
            print("기존에 저장된 지도 파일을 엽니다.")
            webbrowser.open('file://' + existing_file)
        else:
            print("지도 파일이 없습니다.")

경로 시각화 완료! pedestrian_route.html 파일을 확인하세요.
